In [ ]:
import os
import re
import kagglehub

import pandas as pd
from sentence_transformers import SentenceTransformer, util

c:\Users\devas\OneDrive\Desktop\My\Projects\applied knowledge\project_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


data preprocessing

In [2]:
def removal(text):
    return re.sub(
        r"\s+", " ",
        re.sub(
            r"[^a-z\s]", " ",
            re.sub(
                r"http\S+|www\S+|<.*?>", " ",
                text.lower()
    ))).strip()

In [ ]:
path = kagglehub.dataset_download("mantunes/semantic-corpus-from-web-search-snippets")
print("Path to dataset files:", path)

data_file = os.path.join(path, "dataset", "dataset", "xbox.csv")

Path to dataset files: C:\Users\devas\.cache\kagglehub\datasets\mantunes\semantic-corpus-from-web-search-snippets\versions\1


In [4]:
lines = []
with open(data_file, "r") as f:
    lines = f.readlines()

allowed = set("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789")
filtered = [line.strip() for line in lines if any(ch in allowed for ch in line)]

df = pd.DataFrame(filtered, columns=['text'])
text = df['text'].apply(removal)

model training

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [11]:
def get_hits(text, query, model=model):
    text_embs = model.encode(
        text,
        convert_to_tensor=True
    )
    vec = model.encode(
        query,
        convert_to_tensor=True,
    )
    return util.semantic_search(
        vec,
        text_embs,
        top_k=3,
    )

In [16]:
hits = get_hits(text, "how to join")
for hit in hits[0]:
    print(f"{text[hit['corpus_id']]} --> score: {hit['score'] * 100:.4f}/100")

how to install --> score: 40.8212/100
do you already have an account --> score: 34.0768/100
no create an account now --> score: 33.8187/100


evaluation

deployment

In [18]:
model.save('./hit_model')